# Calcul du nombre de prélèvements CVM non conforme par commune et par année

L'objectif de ce notebook est de partir de la liste de communes cog_communes, et pour chaque commune et chaque année, calculer le nombre de prélèvements non conformes pour le CVM.

Il y aura plusieurs aggrégations à faire :

- commune (inseecommune) peut avoir plusieurs UDIs (cdreseau) **ET** un UID peut avoir plusieurs communes (inseecommune) 
- un prélèvement (referenceprel) peut être rattaché à plusieurs UDIs (cdreseau)
- un prélèvement (referenceprel) peut être composé de plusieurs paramètres (cdparametresiseeaux) ; mais dans le cas du CVM, il y a un seul paramètre selon la catégorisation de Pauline, donc c'est plus simple




In [1]:
%load_ext sql
%sql duckdb:///../../database/data.duckdb
%config SqlMagic.displaylimit = 10

The 'toml' package isn't installed. To load settings from pyproject.toml or ~/.jupysql/config, install with: pip install toml

Connecting to 'duckdb:///../../database/data.duckdb'

### Les communes

In [2]:
%%sql 
WITH
udi AS (
    SELECT
      de_partition AS year,
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
    GROUP BY 
    1,2,3
),

cog AS (
    SELECT
      DEP AS code_departement,
      REG AS code_region,
      COM AS commune_code_insee,
    FROM 
      cog_communes
)
    SELECT
      udi.commune_code_insee ,
      udi.cdreseau,
      udi.year,
      cog.code_departement,
      cog.code_region
    FROM
      udi
    LEFT JOIN 
      cog
    ON 
      udi.commune_code_insee = cog.commune_code_insee

Running query in 'duckdb:///../../database/data.duckdb'

commune_code_insee,cdreseau,year,code_departement,code_region
01001,001000556,2020,01,84
01002,001000369,2023,01,84
01004,001000249,2020,01,84
01005,001000850,2022,01,84
01006,001000235,2021,01,84
01007,001000003,2022,01,84
01008,001000254,2024,01,84
01009,001000338,2020,01,84
01010,001000260,2022,01,84
01011,001000870,2020,01,84


#### Pour chaque catégorie et années, on veut la liste complète des commmunes

In [3]:
%%sql  
WITH
annees AS (
    SELECT unnest(generate_series(2020, 2024)) as annee
    ),

cat AS (
    SELECT categorie FROM int__mapping_category_simple GROUP BY 1 
    )

SELECT
 annee, categorie
FROM 
annees
CROSS JOIN
cat

Running query in 'duckdb:///../../database/data.duckdb'

annee,categorie
2020,métabolite de pesticide
2020,hydrocarbure
2020,pcb
2020,dioxine et furane
2020,médicament
2020,minéral
2021,métabolite de pesticide
2021,hydrocarbure
2021,pcb
2021,dioxine et furane


In [4]:
%%sql 
WITH
annees AS (
    SELECT unnest(generate_series(2020, 2024)) as annee
    ),

cat AS (
    SELECT categorie FROM int__mapping_category_simple GROUP BY 1 
    ),

year_cat AS (   
    SELECT
     annee, categorie
    FROM 
    annees
    CROSS JOIN
    cat 
),

udi AS (
    SELECT
      de_partition AS year,
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
    GROUP BY 
    1,2,3
),

cog AS (
    SELECT
      DEP AS code_departement,
      REG AS code_region,
      COM AS commune_code_insee,
    FROM 
      cog_communes
),

udi_cog AS (
    SELECT
      udi.commune_code_insee ,
      udi.cdreseau,
      udi.year,
      cog.code_departement,
      cog.code_region
    FROM
      udi
    LEFT JOIN 
      cog
    ON 
      udi.commune_code_insee = cog.commune_code_insee
)
    
SELECT DISTINCT
    annee, 
    categorie,
    commune_code_insee ,
    cdreseau,
    code_departement,
    code_region
 FROM
      udi_cog
 FULL OUTER JOIN
       year_cat
 ON
    udi_cog.year = year_cat.annee

Running query in 'duckdb:///../../database/data.duckdb'

annee,categorie,commune_code_insee,cdreseau,code_departement,code_region
2022,nitrite,01005,001000850,01,84
2020,nitrite,01035,001001219,01,84
2021,nitrite,01049,001000761,01,84
2022,nitrite,01240,001000527,01,84
2022,nitrite,01279,001000395,01,84
2021,nitrite,01386,001000382,01,84
2022,nitrite,02065,002000091,02,32
2021,nitrite,02097,002000106,02,32
2022,nitrite,02422,002000658,02,32
2021,nitrite,02450,002000878,02,32


### Les résultats

#### mesures_cat

In [5]:
%%sql
WITH
resultats AS (
    SELECT
      referenceprel,
      cdparametresiseeaux,
      valtraduite,
      limitequal,
      CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
      regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    FROM  
        edc_resultats 
    )
    
    SELECT
        resultats.*,
        int__mapping_category_simple.categorie
    FROM 
        resultats
    LEFT JOIN 
        int__mapping_category_simple 
    ON
        resultats.cdparametresiseeaux = int__mapping_category_simple .cdparametresiseeaux

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,valtraduite,limitequal,limitequal_float,unite,categorie
00400121459,CDT25,586.0,None,None,None,paramètre organoleptique
00400121459,CHINE,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides
00400121459,CHINOME,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides
00400121459,CHLORB,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides
00400121459,CHLPM,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides
00400121459,CHLX,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides
00400121459,CINOSUL,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides
00400121459,CL,2.8,None,None,None,sous produit désinfection
00400121459,CL2LIB,0.0,None,None,None,sous produit désinfection
00400121459,CLAHA,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides


#### mesures_cat_communes : on associe aux resultats la ref de prelevement et les uid associés

In [6]:
%%sql --save mesures_cat_communes
WITH 
udi AS (
    SELECT
      de_partition AS year,
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
    GROUP BY 
    1,2,3
),
   
prelevement AS (
	SELECT
	  referenceprel,
	  cdreseau,
      dateprel,
	FROM   
      edc_prelevements
),

resultats AS (
    SELECT
      referenceprel,
      cdparametresiseeaux,
      valtraduite,
      limitequal,
      CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
      regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    FROM  
        edc_resultats 
    ),

mesures_cat AS (
    SELECT
        resultats.*,
        int__mapping_category_simple.categorie
    FROM 
        resultats
    LEFT JOIN 
        int__mapping_category_simple 
    ON
        resultats.cdparametresiseeaux = int__mapping_category_simple .cdparametresiseeaux
)
    
    SELECT
        mesures_cat.*,
        prelevement.dateprel,
        udi.commune_code_insee AS commune_code_insee,
    FROM 
        mesures_cat 
    LEFT JOIN
    	prelevement
    ON
       mesures_cat.referenceprel = prelevement.referenceprel
    LEFT JOIN
        udi
    ON
    	udi.cdreseau = prelevement.cdreseau
        AND udi.year = extract( YEAR FROM prelevement.dateprel)

Running query in 'duckdb:///../../database/data.duckdb'

referenceprel,cdparametresiseeaux,valtraduite,limitequal,limitequal_float,unite,categorie,dateprel,commune_code_insee
00100119085,12DCLE,0.0,<=3 µg/L,3.0,µg/L,hydrocarbure,2020-02-14,01333
00100119085,ACTIK40,0.034,None,None,None,radioactivité,2020-02-14,01333
00100119085,ACTITR,0.0,None,None,None,radioactivité,2020-02-14,01333
00100119085,ADET,0.013,"<=0,1 µg/L",0.10000000149011612,µg/L,métabolite de pesticide,2020-02-14,01333
00100119085,ADET2,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,métabolite de pesticide,2020-02-14,01333
00100119085,ADETD,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,métabolite de pesticide,2020-02-14,01333
00100119085,ADSP,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,métabolite de pesticide,2020-02-14,01333
00100119085,ALTMICR,0.0,None,None,None,minéral,2020-02-14,01333
00100119085,AMTH,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides,2020-02-14,01333
00100119085,ATRZ,0.0,"<=0,1 µg/L",0.10000000149011612,µg/L,pesticides,2020-02-14,01333


#### mesures_cat_communes_year

In [7]:
%%sql --with mesures_cat_communes --save mesures_cat_communes_year

SELECT 
 extract( YEAR FROM mesures_cat_communes.dateprel) as annee,
 categorie,
 commune_code_insee,
 SUM(1) AS nb_analyses,
 SUM(case
        when limitequal_float is not NULL and valtraduite >= limitequal_float then 1
        else 0
    end ) AS nb_analyses_not_ok,
 SUM(case
        when limitequal_float is not NULL and valtraduite < limitequal_float then 1
        else 0
    end ) AS nb_analyses_ok,    
FROM
 mesures_cat_communes
GROUP BY 
    1,2,3

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

annee,categorie,commune_code_insee,nb_analyses,nb_analyses_not_ok,nb_analyses_ok
2020,non classé,02728,182,0,35
2020,minéral,02255,25,0,1
2020,non classé,02168,190,0,39
2020,paramètre organoleptique,02774,112,0,0
2020,sous produit désinfection,02353,33,0,11
2020,microbio,02685,61,24,0
2020,sous produit désinfection,02329,38,0,11
2020,nitrite,02312,12,0,12
2020,non classé,02524,190,0,39
2020,paramètre organoleptique,02166,379,0,0


#### mesures_cat_communes_year_cvm

In [8]:
%%sql --with mesures_cat_communes_year --save mesures_cat_communes_year_cvm
SELECT
    annee,
    commune_code_insee,
    CASE WHEN nb_analyses IS NULL OR nb_analyses = 0 THEN 'Pas recherché'
         WHEN nb_analyses_ok =0 THEN 'jamais quantifié'
         WHEN nb_analyses_ok >0 THEN '<= 0,5 µg/L'
         WHEN nb_analyses_not_ok >1 THEN '> 0,5 µg/L'
    END AS resultat
FROM  
 mesures_cat_communes_year
WHERE
 categorie = 'cvm'

Running query in 'duckdb:///../../database/data.duckdb'

annee,commune_code_insee,resultat
2020,01034,"<= 0,5 µg/L"
2020,01193,"<= 0,5 µg/L"
2020,01311,"<= 0,5 µg/L"
2020,01036,"<= 0,5 µg/L"
2020,01361,"<= 0,5 µg/L"
2020,01027,"<= 0,5 µg/L"
2020,01431,"<= 0,5 µg/L"
2020,01225,"<= 0,5 µg/L"
2020,01188,"<= 0,5 µg/L"
2020,01175,"<= 0,5 µg/L"


**Pour un cas plus générique il faudra ajouter une condition sur categorie dans le CASE WHEN resultat**

#### mesures_cat_communes_year_cvm + Joint list annee_cat_communes list

In [9]:
%%sql --save mesures_cat_communes_year_cvm
WITH 
annees AS (
    SELECT unnest(generate_series(2020, 2024)) as annee
    ),

cat AS (
    SELECT categorie FROM int__mapping_category_simple GROUP BY 1 
    ),

year_cat AS (   
    SELECT
     annee, categorie
    FROM 
    annees
    CROSS JOIN
    cat 
),

udi AS (
    SELECT
      de_partition AS year,
      inseecommune AS commune_code_insee,
      cdreseau,
    FROM
      edc_communes
    GROUP BY 
    1,2,3
),

cog AS (
    SELECT
      DEP AS code_departement,
      REG AS code_region,
      COM AS commune_code_insee,
    FROM 
      cog_communes
),

udi_cog AS (
    SELECT
      udi.commune_code_insee ,
      udi.cdreseau,
      udi.year,
      cog.code_departement,
      cog.code_region
    FROM
      udi
    LEFT JOIN 
      cog
    ON 
      udi.commune_code_insee = cog.commune_code_insee
),

LIST_REF_UDI_YEAR AS (    
    SELECT DISTINCT
        annee, 
        categorie,
        commune_code_insee ,
        cdreseau,
        code_departement,
        code_region
     FROM
          udi_cog
     FULL OUTER JOIN
           year_cat
     ON
        udi_cog.year = year_cat.annee
),

   
prelevement AS (
	SELECT
	  referenceprel,
	  cdreseau,
      dateprel,
	FROM   
      edc_prelevements
),

resultats AS (
    SELECT
      referenceprel,
      cdparametresiseeaux,
      valtraduite,
      limitequal,
      CAST(regexp_extract(REPLACE(limitequal, ',', '.'), '-?\d+(\.\d+)?') AS FLOAT) AS limitequal_float,
      regexp_extract(limitequal, '[a-zA-Zµg]+/?[a-zA-Z/L]+$') AS unite,
    FROM  
        edc_resultats 
    ),

mesures_cat AS (
    SELECT
        resultats.*,
        int__mapping_category_simple.categorie
    FROM 
        resultats
    LEFT JOIN 
        int__mapping_category_simple 
    ON
        resultats.cdparametresiseeaux = int__mapping_category_simple .cdparametresiseeaux
),

 mesures_cat_communes AS (
    SELECT
        mesures_cat.*,
        prelevement.dateprel,
        udi_cog.commune_code_insee AS commune_code_insee,
    FROM 
        mesures_cat 
    LEFT JOIN
    	prelevement
    ON
       mesures_cat.referenceprel = prelevement.referenceprel
    LEFT JOIN
        udi_cog
    ON
    	udi_cog.cdreseau = prelevement.cdreseau
    ),

 mesures_cat_communes_year AS(
    SELECT 
     extract( YEAR FROM mesures_cat_communes.dateprel) as annee,
     categorie,
     commune_code_insee,
     SUM(1) AS nb_analyses,
     SUM(case
            when limitequal_float is not NULL and valtraduite >= limitequal_float then 1
            else 0
        end ) AS nb_analyses_not_ok,
     SUM(case
            when limitequal_float is not NULL and valtraduite < limitequal_float then 1
            else 0
        end ) AS nb_analyses_ok,    
    FROM
     mesures_cat_communes
    GROUP BY 
        1,2,3
    )

SELECT
    LIST_REF_UDI_YEAR.annee,
    LIST_REF_UDI_YEAR.commune_code_insee,
    LIST_REF_UDI_YEAR.categorie,
    CASE WHEN coalesce(nb_analyses,0) = 0 THEN 'Pas recherché'
         WHEN coalesce(nb_analyses_ok,0) =0 THEN 'jamais quantifié'
         WHEN coalesce(nb_analyses_ok,0) >0 THEN '<= 0,5 µg/L'
         WHEN coalesce(nb_analyses_not_ok,0) >1 THEN '> 0,5 µg/L'
    END AS resultat
FROM  
 LIST_REF_UDI_YEAR
LEFT JOIN 
 mesures_cat_communes_year
ON
    LIST_REF_UDI_YEAR.annee =  mesures_cat_communes_year.annee
    AND LIST_REF_UDI_YEAR.categorie =  mesures_cat_communes_year.categorie
    AND  LIST_REF_UDI_YEAR.commune_code_insee  =  mesures_cat_communes_year.commune_code_insee
WHERE
 LIST_REF_UDI_YEAR.categorie = 'cvm'

Your query is using one or more of the following snippets: mesures_cat_communes_year, mesures_cat_communes. JupySQL does not support snippet expansion within CTEs yet, CTE generation is disabled

Running query in 'duckdb:///../../database/data.duckdb'

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

annee,commune_code_insee,categorie,resultat
2020,29025,cvm,"<= 0,5 µg/L"
2020,29083,cvm,"<= 0,5 µg/L"
2020,29274,cvm,"<= 0,5 µg/L"
2020,29298,cvm,"<= 0,5 µg/L"
2020,29029,cvm,"<= 0,5 µg/L"
2020,29065,cvm,"<= 0,5 µg/L"
2020,29252,cvm,"<= 0,5 µg/L"
2020,29035,cvm,"<= 0,5 µg/L"
2020,29076,cvm,"<= 0,5 µg/L"
2020,29202,cvm,"<= 0,5 µg/L"


# Check

In [10]:
%%sql
SELECT
    de_partition as year,
    COUNT(DISTINCT inseecommune)
FROM 
 edc_communes
GROUP BY 
    1 
ORDER BY 
   1

Running query in 'duckdb:///../../database/data.duckdb'

year,count(DISTINCT inseecommune)
2020,34788
2021,34833
2022,34874
2023,34852
2024,34809


In [11]:
%%sql --with mesures_cat_communes_year_cvm  
 SELECT 
    annee,
    COUNT(DISTINCT commune_code_insee)
 FROM
   mesures_cat_communes_year_cvm
GROUP BY 
    1

Running query in 'duckdb:///../../database/data.duckdb'

annee,count(DISTINCT commune_code_insee)
2020,34788
2022,34874
2024,34809
2021,34833
2023,34852


In [14]:
%%sql --with mesures_cat_communes_year_cvm  
 SELECT 
   *
 FROM
   mesures_cat_communes_year_cvm
WHERE
    commune_code_insee = '07194'
ORDER BY 
    annee

Running query in 'duckdb:///../../database/data.duckdb'

annee,commune_code_insee,categorie,resultat
2020,07194,cvm,"<= 0,5 µg/L"
2020,07194,cvm,"<= 0,5 µg/L"
2021,07194,cvm,"<= 0,5 µg/L"
2021,07194,cvm,"<= 0,5 µg/L"
2022,07194,cvm,"<= 0,5 µg/L"
2022,07194,cvm,"<= 0,5 µg/L"
2023,07194,cvm,"<= 0,5 µg/L"
2023,07194,cvm,"<= 0,5 µg/L"
2024,07194,cvm,"<= 0,5 µg/L"
2024,07194,cvm,"<= 0,5 µg/L"


**Ici on a des années en double**

In [15]:
%%sql --with mesures_cat_communes_year 
 SELECT 
   *
 FROM
   mesures_cat_communes_year
WHERE
    commune_code_insee = '07194'
    AND categorie = 'cvm'
ORDER BY 
    annee

Running query in 'duckdb:///../../database/data.duckdb'

annee,categorie,commune_code_insee,nb_analyses,nb_analyses_not_ok,nb_analyses_ok
2020,cvm,07194,3,0,3
2021,cvm,07194,3,0,3
2022,cvm,07194,11,8,3
2023,cvm,07194,16,11,5
2024,cvm,07194,10,5,5
